# SimpleElastix tutorial
[SimpleElastix](https://simpleelastix.github.io/) is a robust tool for image registration which tends to work well out-of-the-box.


### Installation
To use it in python, install [SimpleITK-SimpleElastix](https://pypi.org/project/SimpleITK-SimpleElastix/). It is the normal SimpleITK, but built with SimpleElastix additionally enabled. Note that installing this will not uninstall normal SimpleITK, this is usually fine but if you get exceptions when importing SimpleElastix related functionality, try uninstalling SimpleITK before installing SimpleITK-SimpleElastix build.


In [ ]:
#!/usr/bin/env python
import mrid
import sys
import os
import SimpleITK as sitk


def command_iteration(method):
    """ Callback invoked when the optimization has an iteration. """
    print(
        f"{method.GetOptimizerIteration():3} "
        + f"= {method.GetMetricValue():10.5f} "
        + f": {method.GetOptimizerPosition()}"
    )



fixed = sitk.ReadImage(mrid.tositk("/var/mnt/issd/files 2/programming/projects/CTA_aneurysms/image2/CT.nii.gz"), sitk.sitkFloat32)
moving = sitk.ReadImage(mrid.tositk(mrid.get_sri24("T1")), sitk.sitkFloat32)

numberOfBins = 24
samplingPercentage = 0.10

R = sitk.ImageRegistrationMethod()
R.SetMetricAsMattesMutualInformation(numberOfBins)
R.SetMetricSamplingPercentage(samplingPercentage, sitk.sitkWallClock)
R.SetMetricSamplingStrategy(R.RANDOM)
R.SetOptimizerAsRegularStepGradientDescent(1.0, 0.001, 200)
R.SetInitialTransform(sitk.TranslationTransform(fixed.GetDimension()))
R.SetInterpolator(sitk.sitkLinear)

R.AddCommand(sitk.sitkIterationEvent, lambda: command_iteration(R))

outTx = R.Execute(fixed, moving)

print("-------")
print(outTx)
print(f"Optimizer stop condition: {R.GetOptimizerStopConditionDescription()}")
print(f" Iteration: {R.GetOptimizerIteration()}")
print(f" Metric value: {R.GetMetricValue()}")

sitk.WriteTransform(outTx, args[3])

resampler = sitk.ResampleImageFilter()
resampler.SetReferenceImage(fixed)
resampler.SetInterpolator(sitk.sitkLinear)
resampler.SetDefaultPixelValue(100)
resampler.SetTransform(outTx)

out = resampler.Execute(moving)
simg1 = sitk.Cast(sitk.RescaleIntensity(fixed), sitk.sitkUInt8)
simg2 = sitk.Cast(sitk.RescaleIntensity(out), sitk.sitkUInt8)
cimg = sitk.Compose(simg1, simg2, simg1 // 2.0 + simg2 // 2.0)

